In [1]:
import sys
sys.path.append("..")
from jobs import siamese_job
from input.models.deep_sort_cnn.freeze_model import create_graph as create_deep_sort_cnn_graph
from auxillaries.events import EventAggregator

### Training

In [2]:
observer = EventAggregator()

In [3]:
siamese_job(
    source_path='../input/mars/bbox_train/', 
    model_path='../input/models/deep_sort_cnn/mars-small128.pb', 
    graph_creator=create_deep_sort_cnn_graph,
    batch_size=30,
    num_iter=100,
    num_per_class=4,
    margin=0.2,
    lr=1e-2,
    observer=observer,
)


INFO:tensorflow:Summary name conv2_1/1/Elu:0/activations is illegal; using conv2_1/1/Elu_0/activations instead.
INFO:tensorflow:Summary name conv2_3/1/Elu:0/activations is illegal; using conv2_3/1/Elu_0/activations instead.
INFO:tensorflow:Summary name conv3_1/1/Elu:0/activations is illegal; using conv3_1/1/Elu_0/activations instead.
INFO:tensorflow:Summary name conv3_3/1/Elu:0/activations is illegal; using conv3_3/1/Elu_0/activations instead.
INFO:tensorflow:Summary name conv4_1/1/Elu:0/activations is illegal; using conv4_1/1/Elu_0/activations instead.
INFO:tensorflow:Summary name conv4_3/1/Elu:0/activations is illegal; using conv4_3/1/Elu_0/activations instead.


InvalidArgumentError: Nan in summary histogram for: conv2_1/1/Elu_0/activations
	 [[node conv2_1/1/Elu_0/activations (defined at ../input/models/deep_sort_cnn/freeze_model.py:60)  = HistogramSummary[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"](conv2_1/1/Elu_0/activations/tag, conv2_1/1/Elu)]]

Caused by op 'conv2_1/1/Elu_0/activations', defined at:
  File "/opt/conda/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/opt/conda/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/opt/conda/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/opt/conda/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 505, in start
    self.io_loop.start()
  File "/opt/conda/lib/python3.6/site-packages/tornado/platform/a